# Indexing Bragg diffraction data with *DIALS*

## Introduction

*DIALS* is a program for integrating and scaling Bragg data. It is open source and can be used for both macromolecular and small molecule crystallography. You can read more about it here: https://dials.github.io

For this tutorial, we will use *DIALS* for indexing and geometry refinement. Normally this would be followed by integration and scaling, but we will skip those steps to save time.

For a more complete *DIALS* tutorial, see [Processing in Detail](https://dials.github.io/documentation/tutorials/processing_in_detail_betalactamase.html).

### Tutorial data

Download `insulin_2_1.tar` from Zenodo [doi: 10.5281/zenodo.6536805](https://dx.doi.org/10.5281/zenodo.6536805).

The tar archive will expand to a directory called `images` with subfolders `insulin_2_1`, `insulin_2_bkg`, and `metrology`.

Place the `images` folder in the same directory as this notebook.

### Command-line programs in jupyter

Normally you would run *DIALS* in the command line. We will run command line programs from within the notebook by prefixing them with an exclamation point.

To run the command, click in the cell and press shift-Enter.

In [2]:
!dials.version

DIALS 3.dev
Python 3.9.12
Installed in: /Users/steve/Desktop/erice/lib/python3.9/site-packages/dials


## Import the metadata

The metadata (such as X-ray energy, oscillation width, exposure time, etc) are stored in the cbf image headers. The *dials.integrate* command reads this in and produces a *.expt* file

In [4]:
!dials.import images/insulin_2_1

DIALS (2018) Acta Cryst. D74, 85-97. https://doi.org/10.1107/S2059798317017235
DIALS 3.dev
The following parameters have been modified:

input {
  experiments = <image files>
}

--------------------------------------------------------------------------------
  format: <class 'dxtbx.format.FormatCBFMiniPilatusCHESS_6MSN127.FormatCBFMiniPilatusCHESS_6MSN127'>
  num images: 500
  sequences:
    still:    0
    sweep:    1
  num stills: 0
--------------------------------------------------------------------------------
Writing experiments to imported.expt


The output file *imported.expt* is just a text file (json format). You can open it by double-clicking in the file browser.

*DIALS* also has a method for summarizing expt files

In [7]:
!dials.show imported.expt

DIALS (2018) Acta Cryst. D74, 85-97. https://doi.org/10.1107/S2059798317017235
The following parameters have been modified:

input {
  experiments = imported.expt
}

Experiment 0:
Experiment identifier: 5c372934-4c5d-edaa-d7f3-4378db7454b5
Image template: /Users/steve/Box/lab/desk/steve/projects/mdx2-dev/data/mdx2-v3/images/insulin_2_1/insulin_2_1_####.cbf.bz2
Detector:
Panel:
  name: Panel
  type: SENSOR_PAD
  identifier: 
  pixel_size:{0.172,0.172}
  image_size: {2463,2527}
  trusted_range: {-1,1.0098e+06}
  thickness: 1
  material: Si
  mu: 3.92851
  gain: 1
  pedestal: 0
  fast_axis: {1,0,0}
  slow_axis: {0,-1,0}
  origin: {-217.491,213.713,-260.05}
  distance: 260.05
  pixel to millimeter strategy: ParallaxCorrectedPxMmStrategy
    mu: 3.92851
    t0: 1


Max resolution (at corners): 1.155825
Max resolution (inscribed):  1.485702

Beam:
    wavelength: 0.9768
    sample to source direction : {0,0,1}
    divergence: 0
    sigma divergence: 0
    polarization normal: {0,1,0}
    pol

## DIALS image viewer

We'll use the *DIALS* image viewer to inspect the raw data.

Launch a new terminal window and type:

```bash
dials.image_viewer imported.expt
```

Adjust the brighness slider so that both the Bragg peaks and background scattering are visible.

- Do you see any diffuse scattering?
- Scroll through some images. Is the crystal mosaicity high or low?

## Masking

Some regions of the detector are unreliable. Gaps between panels are automatically masked using *DIALS*. However, we need to make a mask for the beamstop.

This can be done manually in the *DIALS* image viewer. But to save time, we'll do it programatically on the command line.

In [12]:
!dials.generate_mask imported.expt untrusted.circle=1264,1242,50
!dials.apply_mask imported.expt input.mask=pixels.mask output.experiments=imported.expt

The following parameters have been modified:

untrusted {
  circle = 1264 1242 50
}
input {
  experiments = imported.expt
}

DIALS (2018) Acta Cryst. D74, 85-97. https://doi.org/10.1107/S2059798317017235
Writing mask to pixels.mask
The following parameters have been modified:

input {
  mask = "pixels.mask"
  experiments = imported.expt
}
output {
  experiments = "imported.expt"
}

Writing experiments to imported.expt


Launch the dials image viewer again. Click the "show mask" checkbox. You should see the beamstop region masked out.

## Spot finding

Next, *DIALS* will read all of the images and locate Bragg peaks. This can take a while (be patient).

In [14]:
!dials.find_spots imported.expt

DIALS (2018) Acta Cryst. D74, 85-97. https://doi.org/10.1107/S2059798317017235
DIALS 3.dev
The following parameters have been modified:

input {
  experiments = imported.expt
}

Setting spotfinder.filter.min_spot_size=3
Configuring spot finder from input parameters
--------------------------------------------------------------------------------
Finding strong spots in imageset 0
--------------------------------------------------------------------------------

Finding spots in image 1 to 500...
Setting nproc=6
Setting chunksize=83
Extracting strong pixels from images
 Using multiprocessing with 6 parallel job(s)

Found 1600 strong pixels on image 1
Found 1569 strong pixels on image 2
Found 1304 strong pixels on image 3
Found 1484 strong pixels on image 4
Found 1568 strong pixels on image 5
Found 1653 strong pixels on image 6
Found 1766 strong pixels on image 7
Found 1719 strong pixels on image 8
Found 1966 strong pixels on image 9
Found 1949 strong pixels on image 10
Found 1729 strong p

## Indexing

Next we need to assign Miller indices to the spots and to refine the diffraction geometry.

We'll skip the usual space group determination steps by telling *DIALS* the space group

In [16]:
!dials.index imported.expt strong.refl space_group=199

DIALS (2018) Acta Cryst. D74, 85-97. https://doi.org/10.1107/S2059798317017235
DIALS 3.dev
The following parameters have been modified:

indexing {
  known_symmetry {
    space_group = "I 21 3"
  }
}
input {
  experiments = imported.expt
  reflections = strong.refl
}

Found max_cell: 74.2 Angstrom
Setting d_min: 1.45
FFT gridding: (256,256,256)
Number of centroids used: 50261
Candidate solutions:
+-------------------------------------+----------+----------------+------------+-------------+-------------------+-----------+-----------------+-----------------+
| unit_cell                           |   volume |   volume score |   #indexed |   % indexed |   % indexed score |   rmsd_xy |   rmsd_xy score |   overall score |
|-------------------------------------+----------+----------------+------------+-------------+-------------------+-----------+-----------------+-----------------|
| 68.61 68.61 68.61 109.5 109.5 109.5 |   248622 |           0.02 |      48839 |          97 |              0.0

The indexing program produces a lot of output. For diffuse scattering, it's important to verify that all peaks are indexed well. There might be split Bragg reflections, multiple lattices, or salt crystals contaminating the signal. Or, the crystal might have slipped during data collection.

Check that indexing was successful by finding the following parameters at the end of the *dials.index* output, above.

Find the table with `% indexed`. 

* How many strong peaks could not be indexed?

Find the table called `RMSDs by experiment`

* What is the deviation in peak position on the detector?
* What is deviation in frame number, or degrees of oscillation?

## Refinement

*DIALS* can refine a scan-varying model of the crystal geometry to improve the predicted spot locations. 

This can be important if the lattice constants change due to radiation damage, or if the crystal is not held rigidly in the loop.

In [20]:
!dials.refine indexed.expt indexed.refl

DIALS (2018) Acta Cryst. D74, 85-97. https://doi.org/10.1107/S2059798317017235
DIALS 3.dev
The following parameters have been modified:

input {
  experiments = indexed.expt
  reflections = indexed.refl
}

Configuring refiner
Setting outlier.nproc=6

Summary statistics for 50635 observations matched to predictions:
+-------------------+---------+----------+-----------+---------+--------+
|                   |     Min |       Q1 |       Med |      Q3 |    Max |
|-------------------+---------+----------+-----------+---------+--------|
| Xc - Xo (mm)      | -0.3512 | -0.02151 |  0.002323 | 0.02393 | 0.4996 |
| Yc - Yo (mm)      | -0.3488 | -0.02623 |  0.001705 | 0.02626 | 0.3831 |
| Phic - Phio (deg) |  -0.958 | -0.01668 | -0.001109 |  0.0151 |  1.247 |
| X weights         |   251.7 |    386.5 |     401.2 |   404.9 |  405.6 |
| Y weights         |   239.2 |    372.6 |     395.7 |   403.8 |  405.6 |
| Phi weights       |   871.2 |     1199 |      1200 |    1200 |   1200 |
+----------------

Find the table of `RMSDs by experiment`. 

* Did the spot predictions improve?

## Analysis

*DIALS* report contains some useful plots.

In [26]:
!dials.report refined.expt refined.refl

DIALS (2018) Acta Cryst. D74, 85-97. https://doi.org/10.1107/S2059798317017235
The following parameters have been modified:

input {
  experiments = refined.expt
  reflections = refined.refl
}

Analysing strong spots
 Selected 51111 strong reflections.........................................0.02s
Analysing reflection centroids
 Selected 45510 refined reflections........................................0.01s
 Analysing centroid differences with I/Sigma > 0
 Analysing centroid differences in x/y with I/Sigma > 0
 Analysing centroid differences in z with I/Sigma > 0
 Analysing centroid differences vs phi with I/Sigma > 0
Analysing reflection intensities
 Selecting only integrated reflections...Analysing reference profiles
 Skipping: following required fields not present:
  intensity.prf.value
  intensity.prf.variance
  profile.correlation
Analysing scan-varying crystal model
Analysing scaling model
Calculating and generating merging statistics plots
Writing html report to: dials.report.htm

Open the file [dials.report.html](dials.report.html).

Expand the tab called "Analysis of scan varying model".

- Did the unit cell axes change size during data collection?
- Did the crystal slip?

Expand the tab called "Analysis of reflection centroids".

- Is the error in X, Y position random? (hint: the Pilatus 6M has discrete modules)
- Are the spot centroid errors mostly compact, or are there outliers?

I think you will agree that the dataset is fairly ideal, and we can confidently apply the scan-varying model for diffuse mapping.

## Preparing for *mdx2*

For rest of the workshop we'll be using *mdx2*. We need to do two things to prepare:

1. Import background scattering images
2. Prepare a smaller dataset for testing purposes

The experiment was performed at ambient temperature. To maintain crystal hydration, it was inside a plastic capillary, which has significant scattering. To measure this scattering (and also the scattering of air) the crystal was translated out of the beam, and the same wedge of data was acquired (with 1 deg. oscillation rather than 0.1 deg.)

First, import the background images and apply the mask.

In [24]:
!dials.import images/insulin_2_bkg output.experiments=background.expt
!dials.apply_mask background.expt input.mask=pixels.mask output.experiments=background.expt

DIALS (2018) Acta Cryst. D74, 85-97. https://doi.org/10.1107/S2059798317017235
DIALS 3.dev
The following parameters have been modified:

output {
  experiments = "background.expt"
}
input {
  experiments = <image files>
}

--------------------------------------------------------------------------------
  format: <class 'dxtbx.format.FormatCBFMiniPilatusCHESS_6MSN127.FormatCBFMiniPilatusCHESS_6MSN127'>
  num images: 50
  sequences:
    still:    0
    sweep:    1
  num stills: 0
--------------------------------------------------------------------------------
Writing experiments to background.expt
The following parameters have been modified:

input {
  mask = "pixels.mask"
  experiments = background.expt
}
output {
  experiments = "background.expt"
}

Writing experiments to background.expt


Open *background.expt* using *dials.image_viewer*.

* What features do you see in the background scattering?

Finally, we'll make a smaller dataset for testing purposes. The full scan was 50 degrees (500 images). The test set will be 6 degrees (60 images).

In [25]:
!dials.slice_sequence refined.expt "image_range=1 60"

The following parameters have been modified:

image_range = 1 60
input {
  experiments = refined.expt
}

Saving sliced experiments to refined_1_60.expt
